<a href="https://colab.research.google.com/github/ahmedhas32/Data-science-projects/blob/master/warehouse_IVS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyreadstat

In [ ]:
import xgboost
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy.stats import skew
from collections import OrderedDict
from sklearn import preprocessing
from datetime import datetime


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install fsspec>=0.3.3

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_excel('/content/Outbound_alltrips_2018+2019_aggr3.xlsx')

In [ ]:
age= [ 'from_0_to_15_sum',	'from_16_to_25_sum',	'from_26_to_45_sum',	'from_46_to_60_sum',	'more_than_60_sum']

In [ ]:
id_vars = [item for item in list(data.columns)  if item not in age]

In [ ]:
data1 = data.melt( id_vars = id_vars ,value_vars =age ,
        value_name = 'age_groups' , var_name = 'age')

In [ ]:
data1['age_pct'] = data1['age_groups'] / data1['new_wt_sum']

In [ ]:
idx = data1.loc[data1['age_groups'] != 0].index

In [ ]:
data1 = data1.iloc[idx , :]

In [ ]:
gender= ['male_tot_sum' , 'female_tot_sum']

In [ ]:
id_vars = [item for item in list(data1.columns)  if item not in gender]

In [ ]:
data2= data1.melt( id_vars = id_vars ,value_vars =gender ,
        value_name = 'gender_groups' , var_name = 'gender' )

In [ ]:
data2['gender_groups'].sum()

165039519.83879384

In [ ]:
data2['gen_pct'] = data2['gender_groups'] / data2['new_wt_sum']

In [ ]:
idx = data2.loc[data2['gender_groups'] != 0].index

In [ ]:
data2 = data2.iloc[idx , :]

In [ ]:
data2['new_trips'] = data2['gen_pct'] * data2['age_pct']   * data2['new_wt_sum']

In [ ]:
data2['new_nights'] = data2['gen_pct'] * data2['age_pct'] * data2['night_wt_sum']

In [ ]:
data2['new_spend'] = data2['gen_pct'] * data2['age_pct']  * data2['exp_wt_sum']

In [ ]:
data2['new_trips'].sum()

54620912.95207851

In [ ]:
data2

,Year,Q_479N,GRP,Country,Q_12N,Trip_Type,Q_476_O1N,new_pov,Q_652,new_wt_sum,night_wt_sum,exp_wt_sum,age,age_groups,age_pct,gender,gender_groups,gen_pct,new_trips,new_nights,new_spend
0,2019,1,GCC,Bahrain,Saudi,Day trip,NaN,leisure,Married,421250.521643,0.000000,2.542603e+08,from_0_to_15_sum,149091.689034,0.353926,male_tot_sum,312345.902777,0.741473,110547.467161,0.000000,6.672475e+07
1,2019,1,GCC,Bahrain,Saudi,tourist trip,Hotel/ Resorts,leisure,Married,128151.514114,304518.449381,1.680703e+08,from_0_to_15_sum,40697.621933,0.317574,male_tot_sum,90474.908544,0.706000,28732.501897,68275.252032,3.768259e+07
2,2019,1,GCC,Bahrain,Saudi,tourist trip,Furnished Apartment,leisure,Married,31720.671810,58366.036131,4.756713e+07,from_0_to_15_sum,8410.509554,0.265143,male_tot_sum,26427.851143,0.833143,7007.155960,12893.166967,1.050767e+07
3,2019,1,GCC,Bahrain,Saudi,tourist trip,With Friends/ Relatives,VFR,Married,6344.134362,29183.018066,7.409949e+06,from_0_to_15_sum,2030.122996,0.320000,male_tot_sum,5329.072864,0.840000,1705.303317,7844.395256,1.991794e+06
4,2019,1,GCC,Bahrain,Non-Saudi,Day trip,NaN,VFR,Married,39672.000000,0.000000,2.346208e+07,from_0_to_15_sum,9918.000000,0.250000,male_tot_sum,29754.000000,0.750000,7438.500000,0.000000,4.399139e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24756,2018,9,SOUTH EUROPE,Spain,Saudi,tourist trip,Hotel/ Resorts,leisure,Married,561.512232,4772.853972,1.495432e+06,more_than_60_sum,40.108017,0.071429,female_tot_sum,140.378058,0.250000,10.027004,85.229535,2.670414e+04
24758,2018,9,EAST MEDITRANNE EUROPE,Turkey,Saudi,tourist trip,Hotel/ Resorts,leisure,Married,31696.620865,357307.362480,1.459545e+08,more_than_60_sum,419.128871,0.013223,female_tot_sum,9799.632173,0.309170,129.581913,1460.741565,5.966901e+05
24760,2018,11,MIDDLE EAST,Egypt,Saudi,tourist trip,Hotel/ Resorts,others,Married,3243.508033,9730.524100,8.143053e+06,more_than_60_sum,648.701607,0.200000,female_tot_sum,1946.104820,0.600000,389.220964,1167.662892,9.771664e+05
24761,2018,11,MIDDLE EAST,Jordan,Non-Saudi,tourist trip,With Friends/ Relatives,VFR,Married,28396.316667,785245.083334,7.851910e+07,more_than_60_sum,386.344444,0.013605,female_tot_sum,8093.916111,0.285034,110.121308,3045.191262,3.044982e+05


In [ ]:
data['new_wt_sum'].sum()

54620912.952078514

In [ ]:
data2

,Year,Q_479N,GRP,Country,Q_12N,Trip_Type,Q_476_O1N,new_pov,Q_652,new_wt_sum,night_wt_sum,exp_wt_sum,age,age_groups,age_pct,gender,gender_groups,gen_pct,new_trips,new_nights,new_spend
0,2019,1,GCC,Bahrain,Saudi,Day trip,NaN,leisure,Married,421250.521643,0.000000,2.542603e+08,from_0_to_15_sum,149091.689034,0.353926,male_tot_sum,312345.902777,0.741473,110547.467161,0.000000,6.672475e+07
1,2019,1,GCC,Bahrain,Saudi,tourist trip,Hotel/ Resorts,leisure,Married,128151.514114,304518.449381,1.680703e+08,from_0_to_15_sum,40697.621933,0.317574,male_tot_sum,90474.908544,0.706000,28732.501897,68275.252032,3.768259e+07
2,2019,1,GCC,Bahrain,Saudi,tourist trip,Furnished Apartment,leisure,Married,31720.671810,58366.036131,4.756713e+07,from_0_to_15_sum,8410.509554,0.265143,male_tot_sum,26427.851143,0.833143,7007.155960,12893.166967,1.050767e+07
3,2019,1,GCC,Bahrain,Saudi,tourist trip,With Friends/ Relatives,VFR,Married,6344.134362,29183.018066,7.409949e+06,from_0_to_15_sum,2030.122996,0.320000,male_tot_sum,5329.072864,0.840000,1705.303317,7844.395256,1.991794e+06
4,2019,1,GCC,Bahrain,Non-Saudi,Day trip,NaN,VFR,Married,39672.000000,0.000000,2.346208e+07,from_0_to_15_sum,9918.000000,0.250000,male_tot_sum,29754.000000,0.750000,7438.500000,0.000000,4.399139e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24756,2018,9,SOUTH EUROPE,Spain,Saudi,tourist trip,Hotel/ Resorts,leisure,Married,561.512232,4772.853972,1.495432e+06,more_than_60_sum,40.108017,0.071429,female_tot_sum,140.378058,0.250000,10.027004,85.229535,2.670414e+04
24758,2018,9,EAST MEDITRANNE EUROPE,Turkey,Saudi,tourist trip,Hotel/ Resorts,leisure,Married,31696.620865,357307.362480,1.459545e+08,more_than_60_sum,419.128871,0.013223,female_tot_sum,9799.632173,0.309170,129.581913,1460.741565,5.966901e+05
24760,2018,11,MIDDLE EAST,Egypt,Saudi,tourist trip,Hotel/ Resorts,others,Married,3243.508033,9730.524100,8.143053e+06,more_than_60_sum,648.701607,0.200000,female_tot_sum,1946.104820,0.600000,389.220964,1167.662892,9.771664e+05
24761,2018,11,MIDDLE EAST,Jordan,Non-Saudi,tourist trip,With Friends/ Relatives,VFR,Married,28396.316667,785245.083334,7.851910e+07,more_than_60_sum,386.344444,0.013605,female_tot_sum,8093.916111,0.285034,110.121308,3045.191262,3.044982e+05


In [ ]:
data2['new_spend'].sum()

145531871591.51495

In [ ]:
data['exp_wt_sum'].sum()

145531871591.51498

In [ ]:
data2['Q_42'] = data2['Q_42'].apply(lambda x: x[:-5])

KeyError: ignored

In [ ]:
data2.drop (columns = ['new_wt_sum' ,'exp_wt_sum' , 'night_wt_sum' ,'gender_groups' ,'gen_pct'] , inplace=True)

In [ ]:
spend = ['accomodation_spend', 'food_spend',
       'local_transport_spend', 'entertainment_spend', 'shop_spend',
       'medical_spend', 'edu_spend', 'other_spend']

In [ ]:
for i  in spend: 
  data2[i] = data2[i] * data2['per']


In [ ]:
data2['Total Trips'] = data2['new_trips']

In [ ]:
data2['Total Nights'] = data2['new_nights']

In [ ]:
data2['Total Spend'] = data2['new_spend']

In [ ]:
data2.drop(columns =['new_trips' , 'new_nights' , 'new_spend' ,'age_groups' , 'age_pct'] , inplace=True)

In [ ]:
data2.head()

,Year,Q_479N,GRP,Country,Q_12N,Trip_Type,Q_476_O1N,new_pov,Q_652,age,gender,Total Trips,Total Nights,Total Spend
0,2019,1,GCC,Bahrain,Saudi,Day trip,NaN,leisure,Married,from_0_to_15_sum,male_tot_sum,110547.467161,0.000000,6.672475e+07
1,2019,1,GCC,Bahrain,Saudi,tourist trip,Hotel/ Resorts,leisure,Married,from_0_to_15_sum,male_tot_sum,28732.501897,68275.252032,3.768259e+07
2,2019,1,GCC,Bahrain,Saudi,tourist trip,Furnished Apartment,leisure,Married,from_0_to_15_sum,male_tot_sum,7007.155960,12893.166967,1.050767e+07
3,2019,1,GCC,Bahrain,Saudi,tourist trip,With Friends/ Relatives,VFR,Married,from_0_to_15_sum,male_tot_sum,1705.303317,7844.395256,1.991794e+06
4,2019,1,GCC,Bahrain,Non-Saudi,Day trip,NaN,VFR,Married,from_0_to_15_sum,male_tot_sum,7438.500000,0.000000,4.399139e+06


In [ ]:
data2.to_excel('out_2018_2019.xlsx')

In [ ]:
#data = pd.read_spss('/content/drive/My Drive/IVS_alltrips_2018_warehouse.sav')

In [ ]:
#data.head()

In [ ]:
#data1=pd.get_dummies(data , columns=['Q_74','Q_75', 'AGE_NEW' , 'EDU_NEW' ])

In [ ]:
#for i in col : 
 # data1[i] = data1[i]*data1['new_wt']

In [ ]:
#col1 = ["return_visit_Don't Know / Can't Say",
     #  'return_visit_Likely to visit in the next 1 to 3 months',
      # 'return_visit_Likely to visit in the next 1 year',
      # 'return_visit_Likely to visit in the next 3 to 5 years',
      # 'return_visit_Likely to visit in the next 6 months', 'return_visit_Not likely at all',
      # 'satisfied_Extremely dissatisfied', 'satisfied_Extremely satisfied',
      # 'satisfied_Niether satisfied nor dissatisfied', 'satisfied_Somewhat dissatisfied',
      # 'satisfied_Somewhat satisfied', 'AGE_NEW_18-21', 'AGE_NEW_30-34',
      # 'AGE_NEW_40-44', 'AGE_NEW_45-55', 'AGE_NEW_55+',
      # 'EDU_NEW_Post Graduate', 'EDU_NEW_University',
      # 'EDU_NEW_Upto High School']



In [ ]:
# using naive method 
# to convert lists to dictionary 
#res = {} 
#for key in col: 
#    for value in col1: 
#        res[key] = value 
#        col1.remove(value) 
#        break  
  

In [ ]:
#data1.rename(columns=res , inplace=True)

In [ ]:
#df = data[data.Q_13_O1 == 'Bahrain']

In [ ]:
#df.head()

In [ ]:
#df.shape

In [ ]:
#df = df.drop(columns=['male_tot' , 'female_tot' , 'from_0_15_pct_new',	'from_16_25_pct_new',	'from_26_45_pct_new',	'from_46_60_pct_new',	'from_60_pct_new']).groupby(['Trip_Type' , 'Q_20' , 'Q_24N' , 'Q_42' , 'Q_39N' ,'Q_89N1' , 'Q_86' , 'Q_75' , 'AGE_NEW' , 'EDU_NEW'] ).sum().reset_index()

In [ ]:
#df.to_excel('bahrain.xlsx')

In [ ]:
#df

In [ ]:
#df = df[df.new_wt.notnull()]

In [ ]:
#df

In [ ]:
#df.head()

In [ ]:
#df.index

In [ ]:
#df1['variable'].value_counts()

In [ ]:
#df

In [ ]:
#df.to_csv('data.csv')

In [ ]:
#data.melt( id_vars=['Trip_Type', 'Q_20', 'Q_24N', 'Q_42','new_wt' ,'night_wt', 'accom_spend_ind', 'food_spend_ind',
#       'trans_spend_ind', 'enter_spend_ind', 'shop_spend_ind', 'med_spend_ind',
#       'edu_spend_ind', 'other_spend_ind' ] ,value_vars = [ 'from_0_15_pct_new',	'from_16_25_pct_new',	'from_26_45_pct_new',	'from_46_60_pct_new',	'from_60_pct_new'] ,
#        value_name = 'age_groups' , var_name = 'age' )